In [7]:
# notebook setup
import sys
sys.path += ['../']

In [4]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]=".XX"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"]="platform"

import jax.numpy as np
import jax.scipy as jsp
from jax import vmap, jit
from jax.lax import scan

def jordan_block(l, n):
    # Compute a Jordan block for eigenvalue l of size n
    diag_block = np.full(n, l, dtype='float64')
    off_diag_block = np.eye(n, k=1, dtype='float64')[:-1, :]
    return diag_block, off_diag_block

def jordan_blocks(eig_vals, eig_vecs):
    n = eig_vals.shape[0]
    jordan_blocks = []
    i = 0
    while i < n:
        block = jordan_block(eig_vals[i], 1)
        if i < n - 1 and eig_vals[i] == eig_vals[i + 1]:
            block_size = 1
            while i + block_size < n and eig_vals[i + block_size] == eig_vals[i]:
                block_size += 1
            block = jordan_block(eig_vals[i], block_size)
            i += block_size - 1
        jordan_blocks.append(block)
        i += 1
    return jordan_blocks

def jordan_form(x):
    # Compute the Jordan form of a matrix
    eig_vals, eig_vecs = np.linalg.eig(x)
    j_blocks = jordan_blocks(eig_vals, eig_vecs)
    diag_blocks = [block[0] for block in j_blocks]
    off_diag_blocks = [block[1] for block in j_blocks]
    j = jsp.linalg.block_diag(*diag_blocks)
    for i in range(len(off_diag_blocks)):
        if i == 0:
            off_diag = off_diag_blocks[i]
        else:
            off_diag = np.block([[off_diag, None],
                                 [None, off_diag_blocks[i]]])
        blocked_off_diag = np.block([[np.zeros((off_diag.shape[0], j.shape[1] - off_diag.shape[1])),
                                      off_diag],
                                     [np.zeros((j.shape[0] - j.shape[1], j.shape[1]))]])
        j = j + blocked_off_diag
    return j, eig_vecs

# Testing the code with a sample input
x = np.array([
    [-4, -5, -3, 1, -2, 0, 1, -2],
    [4, 7, 3, -1, 3, 0, -1, 2],
    [0, -1, 0, 0, 0, 0, 0, 0],
    [-1, 1, 2, -4, 2, 0, -3, 1],
    [-8, -14, -5, 1, -6, 0, 1, -4],
    [4, 7, 4, -3, 3, -1, -3, 4],
    [2, -2, -2, 5, -3, 0, 4, -1],
    [6, 7, 3, 0, 2, 0, 0, 3]])

j, v = jordan_form(x)
print("Jordan form:\n", j)
print("Eigenvectors:\n", v)


/tmp/ipykernel_2656356/3092158449.py:13: UserWarning: Explicitly requested dtype float64 requested in full is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  diag_block = np.full(n, l, dtype='float64')
/home/cliff/.local/share/virtualenvs/jordan-decomposition-1FTfkCrW/lib/python3.11/site-packages/jax/_src/lax/lax.py:1189: ComplexWarning: Casting complex values to real discards the imaginary part
  fill_value = _convert_element_type(fill_value, dtype, weak_type)
/tmp/ipykernel_2656356/3092158449.py:14: UserWarning: Explicitly requested dtype float64 requested in eye is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  off_d

TypeError: add got incompatible shapes for broadcasting: (8, 8), (0, 8).